In [1]:
#//*** This thread helped with the connection
#//***https://stackoverflow.com/questions/37692780/error-28000-login-failed-for-user-domain-user-with-pyodbc

#//**** Update the ODBC Driver
#//**** https://docs.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-ver15
import pyodbc
import datetime
import json
from datetime import date, timedelta
import pandas as pd
print(pyodbc.drivers())

#//*** Use the whole window in the IPYNB editor
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#//*** Maximize columns and rows displayed by pandas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
server = 'tcp:OM-CASF-DB01' 
server = 'OM-CASF-DLSQL' 
# server = '10.218.97.2'
database = 'DaletDB' 

with open('./ignore_folder/misc.json') as f:
    data = json.loads(f.read())

username = data["user"] 
password = data["password"]
del data
#cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cnxn = pyodbc.connect('Trusted_Connection=yes;DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

['SQL Server', 'ODBC Driver 17 for SQL Server', 'SQL Server Native Client 11.0']


### SQL Query for MOS Active rundowns ###

In [2]:
startDate = date.today() + timedelta(days=-30)
endDate = date.today() + timedelta(days=30)
query = f"""
select DISTINCT Clocks.clock_date AS Rundown_date_time, titles.title Rundown_Name, MosDevices.Name, STAT.Content Status, Content
from playoutItemsStatus STAT, clocks, MosDevices, titles 
where
    DeviceType='GatewayItemInfo'
and STAT.ClockId = clocks.clock_id 
and clocks.clock_id = titles.title_id
and Clocks.clock_date > '{startDate}'
and Clocks.clock_date <= '{endDate}'
order by Rundown_date_time;
"""
cursor.execute(query)
tables = cursor.fetchall()
tdict={ "ALEX TEST RUNDOWN" : ""}
for row in tables:
    if '<GatewayItemInfo status.l="10">' in row[3]:
        if row[1] not in tdict.keys():
            tdict[row[1]] = ""
            print(datetime.datetime.date(row[0]),row[1])

2023-06-08 5AM Weekday
2023-06-08 6AM Weekday
2023-06-08 7AM CTV
2023-06-18 5AM Sunday
2023-06-23 Promo
2023-06-23 9PM Newscast
2023-06-24 Dalet OD XPression
2023-06-29 11PM WEEKDAY


In [26]:

query = """
select TOP 10 *
from playoutItemsStatus 

"""
cursor.execute(query)
tables = cursor.fetchall()

for row in tables:
    print(row)



    
query = """
SELECT *
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'playoutItemsStatus '
"""
cursor.execute(query)
tables = cursor.fetchall()

for row in tables:
    print(row[2],row[3])

query = """
select *
from playoutItemsStatus 
WHERE DeviceType LIKE 'GatewayItemInfo' and DeviceId = '25'
"""
cursor.execute(query)
tables = cursor.fetchall()

for row in tables:
    print(row)

(29934948, '29934948:1000057151', 'ShowTimeTrackingProgramInfo', 'ShowTimeTracking', '<ShowTimeTrackingProgramInfo actualAiredStartTime.td="2021-03-24T18:31:11.273" onairStatus.l="0" startTrackWithBeginTime.l="0" trackUserId.l="-1" triggerStatus.l="1"/>')
(29934948, '29934948:1000057151:855627', 'ShowTimeTrackingItemInfo', 'ShowTimeTracking', '<ShowTimeTrackingItemInfo airedStartTime.td="2021-03-24T18:31:11.273" automationLock.l="0" currentStory2Offset.tc="P0.000S" isCurrTrackItem.l="0"/>')
(29934948, '29934948:1000057152:855631', 'ShowTimeTrackingItemInfo', 'ShowTimeTracking', '<ShowTimeTrackingItemInfo airedStartTime.td="2021-03-24T18:31:11.273" automationLock.l="0" currentStory2Offset.tc="P0.000S" isCurrTrackItem.l="0"/>')
(29934948, '29934948:1000057152:855632', 'ShowTimeTrackingItemInfo', 'ShowTimeTracking', '<ShowTimeTrackingItemInfo airedDuration.tp="P0.884216S" airedStartTime.td="2021-03-24T18:31:11.273" automationLock.l="0" currentStory2Offset.tc="P0.000S" isCurrTrackItem.l="0

In [4]:
query = """
SELECT *
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'MosDevices'
"""
cursor.execute(query)
tables = cursor.fetchall()
header = ""
print("======================")
print(tables[0][2])
print("======================")
for row in tables:
    header += row[3] + ", "
    
print(header[:-2])
query = """
select *
from MosDevices 
"""
cursor.execute(query)
tables = cursor.fetchall()

for row in tables:
    print(row[0],row[1],row[2],row[3],row[4],row[5],row[6])


MosDevices
Id, Type, Name, Description, HostId, IsIncremental, PreloadByBroadcastServer, Icon, Params
19 22 kgo.pcr2.ovd.mos Ross OD PCR 2 203 1 0
20 22 kgo.pcr3.ovd.mos Ross OD PCR 3 204 1 0
23 22 kgo.pcr2.ovdbu.mos Ross Overdrive PCR2 Backup 205 1 0
25 25 webmos.appdev.kgo.abc.mos WEBMOS Prompter 210 1 0
22 42 xpn.kgo.abc.mos Ross Xpression 201 1 0
14 25 bdl.promptera.kgo.abc.mos Prompter 1 211 1 0
24 22 kgo.pcr3.ovdbu.mos Ross Overdrive PCR3 Backup 206 1 0
15 25 bdl.prompterb.kgo.abc.mos Prompter 2 212 1 0


In [5]:
query = """
SELECT *
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'playoutItemsStatus STAT'
"""
cursor.execute(query)
tables = cursor.fetchall()
header = ""
print("======================")
print(tables)
print("======================")
for row in tables:
    header += row[3] + ", "
    
print(header[:-2])
query = """
select *
from playoutItemsStatus STAT 
"""
cursor.execute(query)
tables = cursor.fetchall()

#//*** Prints lots of Data. Uncomment if needed
#for row in tables:
#    print(row)


[]



In [16]:
query = """
SELECT
  *
FROM
  INFORMATION_SCHEMA.TABLES
"""

cursor.execute(query)
tables = cursor.fetchall()

In [17]:
master_tables = []
for table in tables:
    master_tables.append(table[2])
master_tables.sort()

In [18]:
for table in master_tables:
    print(table)

AD_DomainsPerUser
AD_Groups
AFF_FROM_FORMAT
AFF_FROM_STORAGE
AFF_TO_FORMAT
AFF_TO_STORAGE
AccessRights
AccessibleFormatForUser
ActivationPlacements
ActivationRules
AdditionalLocatorsFromEDLToCopy
AdsGroupsToSync
Affinities
AliasGroups
AliasInAliasGroups
AliasesPools
AliasesPoolsData
Allocation_Policies
ApplicationServerTypes
ApplicationServers
Attributes_Order
AuditTrailActionsConfiguration
AutoSaveData
BPMKeyPerformanceIndicators
BPMTaskListBoardAssociations
BPMTaskListBoardViews
BPMTaskListViews
BPMTaskTypes
BPMTasksGroups
BPMTasksUsers
BlockTemplates
BrokerJobs
BrowserViewRules
BrowserViews
CGTemplates
CRConfiguration
CacheModule
CacheSetting
Callers
Capabilities
CategoriesCodes
CategoryNotifyRules
ChatRoomParticipateGroups
ChatRoomParticipateUsers
ChatRooms
ClassCreationForms
ClassDataDictDateTime
ClassDataDictGlossary
ClassDataDictInteger
ClassDataDictObject
ClassDataDictPeriod
ClassDataDictString
ClassDataDictText
ClassDataDictTimeCode
ClassDataDictTimeCodeDate
ClassDataDictTitle

In [19]:
query = """
SELECT
*
FROM
RundownViews
"""
cursor.execute(query)
results = cursor.fetchall()
for x in results:
    print(x)

(136, '<RundownView id.l="136" viewType.l="1"><blankLineCommentFieldName t="ws">ITEM_Icon</blankLineCommentFieldName><blockTooltipFields t="t"><DisplayedFields ShowResultsAsGrid.b="false"><DisplayedField t="lt"><DisplayedField glossaryLanguageId.l="99" position.l="0" textAttributes.null="" width.l="10"><field t="t"><FieldDef fieldObjectTypes.ll="1 2 4"><displayName t="ws">ITEM_Name</displayName><name t="ws">ITEM_Name</name></FieldDef></field></DisplayedField><DisplayedField glossaryLanguageId.l="99" position.l="1" textAttributes.null="" width.l="10"><field t="t"><FieldDef fieldObjectTypes.ll="1 2"><displayName t="ws">ITEM_ComputedDuration</displayName><name t="ws">ITEM_ComputedDuration</name></FieldDef></field></DisplayedField></DisplayedField></DisplayedFields></blockTooltipFields><deviceTrackSettings t="t"><DeviceTrackingSettings/></deviceTrackSettings><displayedFields t="t"><DisplayedFields ShowResultsAsGrid.b="false" secondFragmentInTimeDisplay.l="0"><DisplayedField t="lt"><Display

In [6]:
query = """
SELECT
*
FROM
StoryContent
WHERE
TitleId='29807928'
"""
cursor.execute(query)
results = cursor.fetchall()
for x in results:
    for y in x:
        print(y)
    

In [31]:
 
query = """
SELECT *
FROM RundownViewsAssociations 
"""
cursor.execute(query)
results = cursor.fetchall()


In [32]:
print ([x[0] for x in cursor.description])
for row in results:
    print(row)

['ObjectId', 'ObjectType', 'RundownViewId', 'Priority', 'DefaultPriority']
('438', 'GROUP-Program', 132, 3, None)
('9', 'GROUP-Program', 132, 3, None)
('438', 'GROUP-Pattern', 137, 1, None)
('9', 'GROUP-Program', 136, 4, None)
('438', 'GROUP-Program', 143, 2, None)
('438', 'GROUP-Program', 135, 5, None)
('9', 'GROUP-Pattern', 141, 0, None)
('9', 'GROUP-Pattern', 138, 2, None)
('438', 'GROUP-Pattern', 141, 3, None)
('438', 'GROUP-Pattern', 143, 2, None)
('9', 'GROUP-Program', 141, 8, None)
('438', 'GROUP-Program', 141, 6, None)
('438', 'GROUP-Planning', 123, 0, None)
('438', 'GROUP-Program', 133, 5, None)
('9', 'GROUP-Program', 133, 4, None)
('166', 'GROUP-Program', 139, 0, None)
('9', 'GROUP-Program', 139, 6, None)
('166', 'GROUP-Program', 138, 1, None)
('166', 'GROUP-Pattern', 138, 0, None)
('438', 'GROUP-Program', 127, 0, None)
('9', 'GROUP-Program', 127, 0, None)
('9', 'GROUP-Program', 140, 7, None)
('9', 'GROUP-Planning', 123, 0, None)
('9', 'GROUP-Program', 138, 5, None)
('438', '

In [9]:
query = """
SELECT * 
FROM INFORMATION_SCHEMA.COLUMNS 
"""
cursor.execute(query)
results = cursor.fetchall()

#for row in results:
#    print(row[3])

In [10]:
titleid_dict = {}
#//*** Script ID 29807928
#//*** Rundown ID 29620400
for row in results:
    if  row[7] == 'int':
        #print(row[2],row[3])
        loop_query = f"""
        SELECT count(*)
        FROM {row[2]}
        WHERE {row[3]}='44145757' 
        """
        cursor.execute(loop_query)
        loop_result = cursor.fetchall()[0][0]
        if loop_result > 0:
            print(row[2],row[3],loop_result)
            titleid_dict[row[2]] = row[3]

Titles_ExFields Title_id 1
TitleDataDictInteger TitleId 44
TitleDataDictString TitleId 32
TitleDataDictDateTime TitleId 3
TitleDataDictText TitleId 2
TitleDataDictPeriod TitleId 3
titles_extend title_id 1
ItemsToBeIndexed Item_id 1
spots title_id 1
titles_in_class title_id 1
objects_in_titles Title_id 1
titles_references title_id 1
titles_mix title_id 1
CreatedModifiedItems Item_id 1
StoryContent TitleId 1
ObjectStatus ObjectId 1
titles title_id 1
StatusLog ObjectId 3


In [17]:
query = """
SELECT * 
FROM INFORMATION_SCHEMA.COLUMNS 
"""
cursor.execute(query)
results = cursor.fetchall()
titleid_dict = {}
#//*** Script ID 29807928
#//*** Rundown ID 29620400
for row in results:
    #print(row)
    if  row[7] == 'int':
        #print(row[2],row[3])
        loop_query = f"""
        SELECT count(*)
        FROM {row[2]}
        WHERE {row[3]}='44145757' 
        """
        cursor.execute(loop_query)
        loop_result = cursor.fetchall()[0][0]
        if loop_result > 0:
            print(row[2],row[3],loop_result)
            titleid_dict[row[2]] = row[3]

Titles_ExFields Title_id 1
TitleDataDictInteger TitleId 44
TitleDataDictInteger NumValue 1
TitleDataDictString TitleId 32
TitleDataDictDateTime TitleId 3
TitleDataDictText TitleId 2
TitleDataDictPeriod TitleId 3
titles_extend title_id 1
ItemsToBeIndexed Item_id 1
spots title_id 1
titles_in_class title_id 1
objects_in_titles Title_id 1
titles_references title_id 1
titles_mix title_id 1
CreatedModifiedItems Item_id 1
StoryContent TitleId 1
ObjectStatus ObjectId 1
titles title_id 1
StatusLog ObjectId 3


In [18]:
for table,column in titleid_dict.items():
        loop_query = f"""
        SELECT *
        FROM {table}
        WHERE {column}='44262124' 
        """
        cursor.execute(loop_query)
        loop_result = cursor.fetchall()
        print(f"{table} -- {column}")
        print ([x for x in cursor.description])
        
        for row in loop_result:
            print(row)
        print("=============")
    

Titles_ExFields -- Title_id
[('Title_id', <class 'int'>, None, 10, 10, 0, False), ('Language_id', <class 'int'>, None, 10, 10, 0, False), ('Topic', <class 'str'>, None, 32, 32, 0, True), ('VersionComment', <class 'str'>, None, 1073741823, 1073741823, 0, True)]
(44262124, 99, ' ', None)
TitleDataDictInteger -- NumValue
[('DataField_Id', <class 'int'>, None, 10, 10, 0, False), ('TitleId', <class 'int'>, None, 10, 10, 0, False), ('ValueOrder', <class 'int'>, None, 10, 10, 0, False), ('NumValue', <class 'int'>, None, 10, 10, 0, False)]
(682, 44305002, 1, 44262124)
(682, 44305003, 1, 44262124)
(682, 44305191, 1, 44262124)
(682, 44305378, 1, 44262124)
(682, 44305384, 1, 44262124)
(682, 44305391, 1, 44262124)
(682, 44305393, 1, 44262124)
TitleDataDictString -- TitleId
[('DataField_Id', <class 'int'>, None, 10, 10, 0, False), ('TitleId', <class 'int'>, None, 10, 10, 0, False), ('ValueOrder', <class 'int'>, None, 10, 10, 0, False), ('StrValue', <class 'str'>, None, 255, 255, 0, False)]
(518, 44

In [30]:
def hunt_db(input_search,**kwargs):
    
    #//**** Assign Default Values
    input_type='int'
    
    display = {
        "items" : True,
    }
    
    #//*** Scan Kwargs and update defaults as needed
    for key,value in kwargs.items():
        if key == "items":
            display[key] = value
        
        if key == "input_type":
            input_type=value

    query = """
    SELECT * 
    FROM INFORMATION_SCHEMA.COLUMNS 
    """
    cursor.execute(query)
    results = cursor.fetchall()
    titleid_dict = {}
    #//*** Script ID 29807928
    #//*** Rundown ID 29620400
    for row in results:
        #print(row[2], row[3])
        if  row[7] == input_type:
            #print(row[2],row[3])
            loop_query = f"""
            SELECT count(*)
            FROM {row[2]}
            WHERE {row[3]}='{input_search}' 
            """
            cursor.execute(loop_query)
            loop_result = cursor.fetchall()[0][0]
            if loop_result > 0:
                print(row[2],row[3],loop_result)
                titleid_dict[row[2]] = row[3]
                
    for table,column in titleid_dict.items():
        if not display['items']:
            if table == 'items':
                print("**************************************")
                print("Skipping Rundown Creation Items.")
                print("Use items=True to display")
                print("**************************************")
                
                continue
            
        loop_query = f"""
        SELECT *
        FROM {table}
        WHERE {column}='{input_search}' 
        """
        cursor.execute(loop_query)
        loop_result = cursor.fetchall()
        print(f"{table} -- {column}")
        out=""
        for desc in cursor.description:
            out += f"{desc[0]}, "
        print(out)
        #print(desc)
        #print ([x for x in cursor.description])
        print("Table")

        
        for row in loop_result:

            out = "( "
            for field in row:
                out += str(field).replace("&gt;",">").replace("&lt;","\n<").replace("&quot;","\"").replace("&#xd;&#xa;","\n").replace("&#xa;&#xa;","\n")
                out += ", "
            out += " )"
            print(out)

        print("=============")

In [20]:
hunt_db("44262124")

Titles_ExFields Title_id 1
TitleDataDictInteger TitleId 44
TitleDataDictInteger NumValue 7
TitleDataDictString TitleId 31
TitleDataDictDateTime TitleId 2
TitleDataDictText TitleId 2
TitleDataDictPeriod TitleId 2
Versions OrginalTitleId 7
titles_extend title_id 1
ItemsToBeIndexed Item_id 1
spots title_id 1
titles_in_class title_id 1
objects_in_titles Title_id 5
titles_references title_id 1
titles_mix title_id 1
CreatedModifiedItems Item_id 1
StoryContent TitleId 1
ObjectStatus ObjectId 1
titles title_id 1
StatusLog ObjectId 3
Titles_ExFields -- Title_id
Title_id, Language_id, Topic, VersionComment, 
Table
( 44262124, 99,  , None,  )
TitleDataDictInteger -- NumValue
DataField_Id, TitleId, ValueOrder, NumValue, 
Table
( 682, 44305002, 1, 44262124,  )
( 682, 44305003, 1, 44262124,  )
( 682, 44305191, 1, 44262124,  )
( 682, 44305378, 1, 44262124,  )
( 682, 44305384, 1, 44262124,  )
( 682, 44305391, 1, 44262124,  )
( 682, 44305393, 1, 44262124,  )
( 682, 44331382, 1, 44262124,  )
TitleDataDi

In [21]:
hunt_db("44262124")

Titles_ExFields Title_id 1
TitleDataDictInteger TitleId 44
TitleDataDictInteger NumValue 8
TitleDataDictString TitleId 31
TitleDataDictDateTime TitleId 2
TitleDataDictText TitleId 2
TitleDataDictPeriod TitleId 2
Versions OrginalTitleId 7
titles_extend title_id 1
ItemsToBeIndexed Item_id 1
spots title_id 1
titles_in_class title_id 1
objects_in_titles Title_id 5
titles_references title_id 1
titles_mix title_id 1
CreatedModifiedItems Item_id 1
StoryContent TitleId 1
ObjectStatus ObjectId 1
titles title_id 1
StatusLog ObjectId 3
Titles_ExFields -- Title_id
Title_id, Language_id, Topic, VersionComment, 
Table
( 44262124, 99,  , None,  )
TitleDataDictInteger -- NumValue
DataField_Id, TitleId, ValueOrder, NumValue, 
Table
( 682, 44305002, 1, 44262124,  )
( 682, 44305003, 1, 44262124,  )
( 682, 44305191, 1, 44262124,  )
( 682, 44305378, 1, 44262124,  )
( 682, 44305384, 1, 44262124,  )
( 682, 44305391, 1, 44262124,  )
( 682, 44305393, 1, 44262124,  )
( 682, 44331382, 1, 44262124,  )
TitleDataDi

In [22]:
hunt_db("44262124")


Titles_ExFields Title_id 1
TitleDataDictInteger TitleId 44
TitleDataDictInteger NumValue 9
TitleDataDictString TitleId 31
TitleDataDictDateTime TitleId 2
TitleDataDictText TitleId 2
TitleDataDictPeriod TitleId 2
Versions OrginalTitleId 7
titles_extend title_id 1
ItemsToBeIndexed Item_id 1
spots title_id 1
titles_in_class title_id 1
objects_in_titles Title_id 5
titles_references title_id 1
titles_mix title_id 1
CreatedModifiedItems Item_id 1
StoryContent TitleId 1
ObjectStatus ObjectId 1
titles title_id 1
StatusLog ObjectId 3
Titles_ExFields -- Title_id
Title_id, Language_id, Topic, VersionComment, 
Table
( 44262124, 99,  , None,  )
TitleDataDictInteger -- NumValue
DataField_Id, TitleId, ValueOrder, NumValue, 
Table
( 682, 44305002, 1, 44262124,  )
( 682, 44305003, 1, 44262124,  )
( 682, 44305191, 1, 44262124,  )
( 682, 44305378, 1, 44262124,  )
( 682, 44305384, 1, 44262124,  )
( 682, 44305391, 1, 44262124,  )
( 682, 44305393, 1, 44262124,  )
( 682, 44331382, 1, 44262124,  )
( 682, 4433

In [24]:
hunt_db("44331403")

Titles_ExFields Title_id 1
TitleDataDictInteger TitleId 11
TitleDataDictString TitleId 6
TitleDataDictText TitleId 1
TitleDataDictPeriod TitleId 2
titles_extend title_id 1
ItemsToBeIndexed Item_id 1
spots title_id 1
titles_in_class title_id 1
objects_in_titles Title_id 5
titles_references title_id 1
titles_mix title_id 1
CreatedModifiedItems Item_id 1
StoryContent TitleId 1
ObjectStatus ObjectId 1
titles title_id 1
StatusLog ObjectId 1
Titles_ExFields -- Title_id
Title_id, Language_id, Topic, VersionComment, 
Table
( 44331403, 99,  , Title Created,  )
TitleDataDictInteger -- TitleId
DataField_Id, TitleId, ValueOrder, NumValue, 
Table
( 63, 44331403, 1, 200,  )
( 64, 44331403, 1, 40,  )
( 67, 44331403, 1, 0,  )
( 68, 44331403, 1, 1,  )
( 69, 44331403, 1, 0,  )
( 71, 44331403, 1, 1,  )
( 73, 44331403, 1, 1,  )
( 76, 44331403, 1, 0,  )
( 78, 44331403, 1, 0,  )
( 682, 44331403, 1, 44262124,  )
( 683, 44331403, 1, 1,  )
TitleDataDictString -- TitleId
DataField_Id, TitleId, ValueOrder, StrVa

In [31]:
#//*** Spots - Block ID
#//*** Items = Ordered List of Item_ID by Specified Block
hunt_db("1000179528")

items block_id 44
spots block_id 34
texts block_id 10
block_item child_block_id 1
blocks block_id 1
items -- block_id
block_id, item_id, item_type, item_reference, scheduled_time, computed_time, item_comment, item_order, radio_id, clock_id, use_manual_duration, 
Table
( 1000179528, 5450111, 1,  , 2023-06-27 23:59:59.940000, 2023-06-27 23:59:59.940000, <Story copyEmbeddedTitlesInApplyPattern.l="1" copyTitleInApplyPattern.l="1" enteredDateTime.td="2021-01-25T20:57:31.471" hold.l="1" mode.l="0" orderPageNumber.l="1" startMode.l="1" tp.l="1"><custom3 t="ws">$$$$1.0</custom3><pageNumber t="ws">A1</pageNumber></Story>, 10000, 1132, 44262093, 0,  )
( 1000179528, 5450112, 1,  , 2023-06-27 23:59:59.940000, 2023-06-27 23:59:59.940000, <Story copyEmbeddedTitlesInApplyPattern.l="1" copyTitleInApplyPattern.l="1" enteredDateTime.td="2021-03-23T18:46:01.519" mode.l="0" orderPageNumber.l="2" plannedDuration.tc="P5.005S" startMode.l="1" tp.l="1"><custom10 t="ws">00:00:10</custom10><custom3 t="ws">5emo<

In [32]:
#//*** Hunt for item ID to get Story ID
hunt_db("5450111")


items item_id 1
spots item_id 1
items -- item_id
block_id, item_id, item_type, item_reference, scheduled_time, computed_time, item_comment, item_order, radio_id, clock_id, use_manual_duration, 
Table
( 1000179528, 5450111, 1,  , 2023-06-27 23:59:59.940000, 2023-06-27 23:59:59.940000, <Story copyEmbeddedTitlesInApplyPattern.l="1" copyTitleInApplyPattern.l="1" enteredDateTime.td="2021-01-25T20:57:31.471" hold.l="1" mode.l="0" orderPageNumber.l="1" startMode.l="1" tp.l="1"><custom3 t="ws">$$$$1.0</custom3><pageNumber t="ws">A1</pageNumber></Story>, 10000, 1132, 44262093, 0,  )
spots -- item_id
block_id, item_id, title_id, mix_type, volume, begin_offset, end_offset, fade_in, fade_out, crossfade, 
Table
( 1000179528, 5450111, 44262113, 0, 0, 0, 0, 0, 0, 0,  )


In [33]:
#//*** Is Clock_ID, Rundown ID? Pulled from Block_id Items

hunt_db("44262093")


CurrentlyOpenedTitles TitleId 1
Titles_ExFields Title_id 1
TitleDataDictInteger TitleId 33
TitleDataDictString TitleId 28
TitleDataDictDateTime TitleId 2
TitleDataDictText TitleId 2
items clock_id 196
resourceReservations clockId 1
titles_extend title_id 1
ItemsToBeIndexed Item_id 1
titles_in_class title_id 1
titles_references title_id 1
titles_mix title_id 1
CreatedModifiedItems Item_id 1
PlayoutItemsStatus ClockId 135
clocks clock_id 1
ObjectStatus ObjectId 1
titles title_id 1
StatusLog ObjectId 1
CurrentlyOpenedTitles -- TitleId
TitleId, UserId, ClientId, UseStart, LastUseReset, 
Table
( 44262093, 2249, 0732c714-59b4-4d61-a89a-75e339b264f7, 2023-06-29 06:13:33.537000, 2023-06-29 06:59:03.703000,  )
Titles_ExFields -- Title_id
Title_id, Language_id, Topic, VersionComment, 
Table
( 44262093, 99, None, None,  )
TitleDataDictInteger -- TitleId
DataField_Id, TitleId, ValueOrder, NumValue, 
Table
( 647, 44262093, 1, 1,  )
( 648, 44262093, 1, 1,  )
( 649, 44262093, 1, 1,  )
( 650, 44262093

In [36]:


            
            
            
#hunt_db('29807933')

#//*** Hunt for BRIO
#hunt_db('29545833')

#//*** StoryID
print("=========================")
print("=========================")
print("=========================")
print("Story - 31527216")
print("=========================")
print("=========================")
print("=========================")
hunt_db('31527216')
#//*** MOS ID Unfloated MOS
print("=========================")
print("=========================")
print("=========================")
print("UnFLoated - 31527217")
print("=========================")
print("=========================")
print("=========================")
hunt_db('31527217')
#//*** MOS ID Floated MOS
print("=========================")
print("=========================")
print("=========================")
print("floated - 31527218")
print("=========================")
print("=========================")
print("=========================")
hunt_db('31527218')
#hunt_db('1173')

Story - 31527216
Titles_ExFields Title_id 1
TitleDataDictInteger TitleId 42
TitleDataDictString TitleId 29
TitleDataDictDateTime TitleId 3
TitleDataDictText TitleId 2
TitleDataDictPeriod TitleId 2
titles_extend title_id 1
ItemsToBeIndexed Item_id 1
spots title_id 1
titles_in_class title_id 1
titles_references title_id 1
titles_mix title_id 1
CreatedModifiedItems Item_id 1
StoryContent TitleId 1
ObjectStatus ObjectId 1
titles title_id 1
StatusLog ObjectId 1
Titles_ExFields -- Title_id
Title_id, Language_id, Topic, VersionComment, 
Table
( 31527216, 99,  , None,  )
TitleDataDictInteger -- TitleId
DataField_Id, TitleId, ValueOrder, NumValue, 
Table
( 67, 31527216, 1, 0,  )
( 68, 31527216, 1, 1,  )
( 69, 31527216, 1, 0,  )
( 71, 31527216, 1, 1,  )
( 73, 31527216, 1, 1,  )
( 76, 31527216, 1, 0,  )
( 78, 31527216, 1, 0,  )
( 114, 31527216, 1, 29557028,  )
( 114, 31527216, 2, 29922070,  )
( 647, 31527216, 1, 1,  )
( 648, 31527216, 1, 1,  )
( 649, 31527216, 1, 1,  )
( 650, 31527216, 1, 1,  )
(

floated - 31527218


In [37]:

#//*** StoryID
print("=========================")
print("=========================")
print("=========================")
print("Story - 12976947")
print("=========================")
print("=========================")
print("=========================")
hunt_db('12976947')

#hunt_db('1173')

Story - 12976947


In [38]:
#select  count(*)
#from playoutItemsStatus, clocks, MosDevices, titles 
#where content like '%onairstatus%' and STAT.ClockId = clocks.clock_id and clocks.clock_id = titles.title_id
#order by Rundown_date_time desc ;
query = """
SELECT *
FROM playoutItemsStatus
"""
cursor.execute(query)
results = cursor.fetchall()
#for x in results:
#    print(x)

    

In [39]:
# Datafields are all the Locking metadata fields. This gets a list of all of them

query = f"""
SELECT *
FROM DataFields 
"""
cursor.execute(query)
results = cursor.fetchall()
#for x in results:
#    print(x)
#titles -- title_id

In [37]:
hunt_db("38122")

items item_order 3
AccessRights Object_id 12
titles_in_class class_id 2058
classes class_id 1
class_tree child_class_id 1
ObjectStatus ObjectId 1
titles duration 2
ObjectAssignedStatus ObjectId 1
StatusLog ObjectId 1
items -- item_order
block_id, item_id, item_type, item_reference, scheduled_time, computed_time, item_comment, item_order, radio_id, clock_id, use_manual_duration, 
( 1000060653, 967984, 3,  , 2021-04-17 11:59:59.980000, 2021-04-17 11:59:59.980000, <Text blankLineColor.l="16777215" blankLineTransparent.b="false" mode.l="0" startMode.l="1"/>, 38122, 1132, 30338778, 0,  )
( 1000066690, 1173908, 1,  , 2021-06-01 22:58:45.937000, 2021-06-01 22:58:45.937000, <Story mode.l="0" orderPageNumber.l="15" startMode.l="1" tp.l="1"><custom10 t="ws">00:00:12 "more affordably"</custom10><custom3 t="ws">cj4p</custom3><pageNumber t="ws">D15</pageNumber></Story>, 38122, 1132, 31067970, 0,  )
( 1000074549, 1473179, 1,  , 2021-08-03 22:58:45.937000, 2021-08-03 22:58:45.937000, <Story mode.l="0

( 38122, 33983778, 1, None,  )
classes -- class_id
class_id, code, long_name, short_name, description, color, n_child, level_code, killdate_offset, sort_field_id1, sort_field_id2, sort_field_id3, sort_direction, access_rights, endkill_offset, titles_mean_duration, record_limitation, full_name, keepdate_offset, SubCategorySorting, is_running_order, read_rate, SystemClass, 
( 38122, SRDC, 2021/11/29, 2021/11/29, 2021/11/29, -1, 0, , 5, 0, 0, 0, 0, 111111111, 1, 210000, 0, CATEGORIES/KGO Rundowns/KGO_NEWS Rundowns/2021/11/29, 0, 0, 0, None, None,  )
class_tree -- child_class_id
parent_class_id, child_class_id, class_order, 
( 31804, 38122, 0,  )
ObjectStatus -- ObjectId
ObjectId, ObjectType, Status_id, 
( 38122, 83, 1,  )
titles -- duration
title_id, title_type_id, title_interface_id, user_id, title, interpret, author, client, duration, is_online, is_recorded, record_date, start_date, end_date, kill_date, audio_duration, use_manual_duration, soundfile_id, year, keywords, site_origin, no_o

In [13]:
hunt_db("33949540")

CurrentlyOpenedTitles TitleId 4
Titles_ExFields Title_id 1
TitleDataDictInteger TitleId 33
TitleDataDictString TitleId 28
TitleDataDictDateTime TitleId 2
TitleDataDictText TitleId 2
items clock_id 136
resourceReservations clockId 1
titles_extend title_id 1
ItemsToBeIndexed Item_id 1
titles_in_class title_id 1
titles_references title_id 1
titles_mix title_id 1
CreatedModifiedItems Item_id 1
PlayoutItemsStatus ClockId 87
clocks clock_id 1
ObjectStatus ObjectId 1
titles title_id 1
StatusLog ObjectId 1
CurrentlyOpenedTitles -- TitleId
TitleId, UserId, ClientId, UseStart, LastUseReset, 
( 33949540, 2082, 7411ff42-b6db-4551-9444-d52eac97540f, 2021-11-30 06:32:52.733000, 2021-11-30 20:02:22.703000,  )
( 33949540, 2123, 4ff4e160-4352-49bf-afd3-69cf74cc3f60, 2021-11-30 19:56:25.070000, 2021-11-30 20:02:25.093000,  )
( 33949540, 2205, 2c5e8eb1-0add-4635-acce-0e7e9aab7286, 2021-11-30 17:32:31.667000, 2021-11-30 20:02:01.637000,  )
( 33949540, 2304, 13546b0d-5b6f-4eb2-b861-b76a7ff6ce6b, 2021-11-30

In [6]:
#//*** Hunt for ParentID. Current theory is this is an ID that represents a Day's
#//*** worth of stories

hunt_db("38122")

items item_order 3
AccessRights Object_id 12
titles_in_class class_id 2052
classes class_id 1
class_tree child_class_id 1
ObjectStatus ObjectId 1
titles duration 2
ObjectAssignedStatus ObjectId 1
StatusLog ObjectId 1
items -- item_order
block_id, item_id, item_type, item_reference, scheduled_time, computed_time, item_comment, item_order, radio_id, clock_id, use_manual_duration, 
( 1000060653, 967984, 3,  , 2021-04-17 11:59:59.980000, 2021-04-17 11:59:59.980000, <Text blankLineColor.l="16777215" blankLineTransparent.b="false" mode.l="0" startMode.l="1"/>, 38122, 1132, 30338778, 0,  )
( 1000066690, 1173908, 1,  , 2021-06-01 22:58:45.937000, 2021-06-01 22:58:45.937000, <Story mode.l="0" orderPageNumber.l="15" startMode.l="1" tp.l="1"><custom10 t="ws">00:00:12 "more affordably"</custom10><custom3 t="ws">cj4p</custom3><pageNumber t="ws">D15</pageNumber></Story>, 38122, 1132, 31067970, 0,  )
( 1000074549, 1473179, 1,  , 2021-08-03 22:58:45.937000, 2021-08-03 22:58:45.937000, <Story mode.l="0

( 38122, 33975673, 1, None,  )
( 38122, 33975676, 1, None,  )
( 38122, 33975679, 1, None,  )
( 38122, 33975683, 1, None,  )
( 38122, 33975687, 1, None,  )
( 38122, 33975699, 1, None,  )
( 38122, 33975720, 1, None,  )
( 38122, 33975726, 1, None,  )
( 38122, 33975738, 1, None,  )
( 38122, 33975747, 1, None,  )
( 38122, 33975749, 1, None,  )
( 38122, 33975751, 1, None,  )
( 38122, 33975789, 1, None,  )
( 38122, 33975840, 1, None,  )
( 38122, 33975842, 1, None,  )
( 38122, 33975899, 1, None,  )
( 38122, 33975901, 1, None,  )
( 38122, 33975928, 1, None,  )
( 38122, 33975940, 1, None,  )
( 38122, 33975942, 1, None,  )
( 38122, 33975957, 1, None,  )
( 38122, 33975966, 1, None,  )
( 38122, 33975968, 1, None,  )
( 38122, 33975980, 1, None,  )
( 38122, 33976040, 1, None,  )
( 38122, 33976057, 1, None,  )
( 38122, 33976060, 1, None,  )
( 38122, 33976071, 1, None,  )
( 38122, 33976127, 1, None,  )
( 38122, 33976140, 1, None,  )
( 38122, 33976143, 1, None,  )
( 38122, 33976223, 1, None,  )
( 38122,

In [35]:
#//**** Hunt: Rundown
hunt_db("44304814",items=True)

Titles_ExFields Title_id 1
TitleDataDictInteger TitleId 36
TitleDataDictString TitleId 32
TitleDataDictDateTime TitleId 3
TitleDataDictText TitleId 2
SOMUpdateHistory Title_id 1
TitleDataDictPeriod TitleId 1
TitleDataDictTimeCodeDate TitleId 1
titles_extend title_id 1
ItemsToBeIndexed Item_id 1
Locators ObjectId 15
LocatorThumbnails ObjectId 3
titles_in_class title_id 2
objects_in_titles Object_id 11
GenealogyRenderedSegments RenderedTitleId 13
GenealogyRenderedSegments SegmentParentTitleId 1
GenealogySegmentAMfields RenderedTitleId 1
GenealogySegmentAMfields SegmentParentTitleId 1
titles_references title_id 1
titles_mix title_id 1
CreatedModifiedItems Item_id 1
Title_Files Title_id 7
ObjectStatus ObjectId 1
titles title_id 1
StatusLog ObjectId 4
Titles_ExFields -- Title_id
Title_id, Language_id, Topic, VersionComment, 
Table
( 44304814, 99, None, None,  )
TitleDataDictInteger -- TitleId
DataField_Id, TitleId, ValueOrder, NumValue, 
Table
( 291, 44304814, 1, 0,  )
( 442, 44304814, 1, 2

In [517]:
#//*** Table: titles
#//*** Search Titles by Date to get Rundown IDs
#//*** Titles Contains All titled Objects including scripts rundowns and MOS objects
#//*** title_type_id = 25 <--- Rundown Objects
#//*** duration > 900000 <--- Rundowns Longer than 15 minutes (15 * 60 Seconds * 1000 ms)
#//***                        Automatically filters out short content like cutins
title_cols = ['title_id','title_type_id','title','duration','start_date']
import datetime


tgt_year = 2021
q1 = f"'{tgt_year}-01-01T00:00:00' AND '{tgt_year}-03-31T23:59:59'"
q2 = f"'{tgt_year}-04-01T00:00:00' AND '{tgt_year}-06-30T23:59:59'"
q3 = f"'{tgt_year}-07-01T00:00:00' AND '{tgt_year}-09-30T23:59:59'"
q4 = f"'{tgt_year}-09-01T00:00:00' AND '{tgt_year}-12-31T23:59:59'"

query = f"""
SELECT *
FROM titles 
WHERE title_type_id = 25 
    AND duration > 900000
    AND start_date BETWEEN {q3}
    AND title <> 'Prepak'
    AND title <> 'Tool Kit'
    AND title <> 'Tricaster'
    AND title <> 'PRODUCER HOLD'
    AND title <> 'Promo'
    AND title <> 'Breaking News'
    AND title <> 'Dalet OD XPression'
    AND title <> 'PRACTICE'
    AND title <> '7AM DIGITAL'
    --AND title <> '7:23AM Cutin'
    --AND title <> '7:56AM Cutin'
    --AND title <> '8:27AM Cutin'
    --AND title <> '8:56AM Cutin'
    --AND title <> '12:25PM Cutin'
    --AND title <> '7:27 Cut-In Weekend AM'
    --AND title <> '7:56AM Cut-In Weekend'
    --AND title <> '8:27AM Cut-In Saturdau'
--WHERE title_type_id = 25 AND start_date BETWEEN '2021-11-01T00:00:00' AND '2021-11-29T23:59:59'
--WHERE start_date BETWEEN '2021-01-24T00:00:00' AND '2021-01-24T23:59:59'
"""



#'2012-01-01T00:00:00'
cursor.execute(query)
results = cursor.fetchall()
#results = cursor.fetchmany(100)
#for x in results:
#    print(x)
#titles -- title_id
#print(dir(cursor))

all_rundowns_df = pd.read_sql(query,cnxn)
print(all_rundowns_df.columns)
all_rundowns_df = all_rundowns_df[title_cols]

#//*** Filter out shows that DO NOT USE
all_rundowns_df = all_rundowns_df[all_rundowns_df['title'].str.contains('DO NOT USE')==False]

#//*** Convert Start_Date to Date only
all_rundowns_df['start_date'] = all_rundowns_df['start_date'].apply(lambda x: x.date())

all_dates = all_rundowns_df['start_date'].unique()

#//*** Pick one day to test with
one_day_df = all_rundowns_df[all_rundowns_df['start_date'] == all_dates[-3]]


print("Quarterly Shows : ", len(all_rundowns_df))
print("Quarterly Show Hours: ", all_rundowns_df['duration'].sum() / 60000 / 60)
all_rundowns_df.iloc[-40:]


Index(['title_id', 'title_type_id', 'title_interface_id', 'user_id', 'title',
       'interpret', 'author', 'client', 'duration', 'is_online', 'is_recorded',
       'record_date', 'start_date', 'end_date', 'kill_date', 'audio_duration',
       'use_manual_duration', 'soundfile_id', 'year', 'keywords',
       'site_origin', 'no_overwrite', 'is_rotational_cart', 'replacement',
       'last_words', 'is_opener', 'weight', 'beats_pm', 'master_record',
       'compagny_id', 'compagny_disp_name', 'album_disp_name', 'package_id',
       'album_id', 'song_id', 'day_part_rest_id', 'keep_date'],
      dtype='object')
Quarterly Shows :  776
Quarterly Show Hours:  589.7785052777778


,title_id,title_type_id,title,duration,start_date
740,32964871,25,11PM WEEKDAY,2113044,2021-09-26
741,32965154,25,3PM Getting Answers,1799998,2021-09-26
742,32965282,25,4PM Weekday,3674003,2021-09-26
743,32965867,25,5PM Weekday,1680011,2021-09-26
744,32966277,25,6PM Weekday,3515979,2021-09-26
745,32969017,25,11PM WEEKEND,3388918,2021-09-26
746,32972732,25,5AM Weekday,3435031,2021-09-27
747,32973519,25,6AM Weekday,3630093,2021-09-27
748,32974613,25,11AM Midday,1728993,2021-09-27
749,32974841,25,11:30AM Midday,1705970,2021-09-27


In [518]:
#//*** Pick one day to test with
one_day_df = all_rundowns_df[all_rundowns_df['start_date'] == all_dates[-3]]

one_day_df

,title_id,title_type_id,title,duration,start_date
755,32991189,25,5AM Weekday,3435031,2021-09-28
756,32991966,25,6AM Weekday,3630093,2021-09-28
757,32993066,25,11AM Midday,1728993,2021-09-28
758,32993292,25,11:30AM Midday,1705970,2021-09-28
759,32999966,25,11PM WEEKDAY,2113044,2021-09-28
760,33000261,25,3PM Getting Answers,1799998,2021-09-28
761,33000392,25,4PM Weekday,3674003,2021-09-28
762,33001020,25,5PM Weekday,1680011,2021-09-28
763,33001453,25,6PM Weekday,3515979,2021-09-28


In [519]:
qtr_df = pd.DataFrame()
for date in all_rundowns_df['start_date'].unique():
    one_day_df = all_rundowns_df[all_rundowns_df['start_date'] == date]
    print(date, "/", all_rundowns_df['start_date'].unique()[-1])
    for row in one_day_df.iterrows():
        loop_rundown = row[1]
        loop_title_id = loop_rundown['title_id']
        #print("Rundown ID: ", loop_title_id, loop_rundown['title'])

        #print(loop_rundown)
        #//*** Get the Blocks associated with the Selected Rundown
        query = f"""
        SELECT block_id
        FROM items 

        WHERE clock_id = '{loop_title_id}'
        """
        loop_blocks = pd.read_sql(query,cnxn)['block_id'].unique()

        #//*** Find the A-Block of Selected Rundown
        #//**** Block Query gets just the A BLOCK from the Rundown Block
        #//*** Combines all Blocks into a single query and returns only the A-Block
        block_query = ""
        for block_id in loop_blocks:
            if block_id == loop_blocks[0]:
                block_query += f"(block_id='{block_id}' "
            else:
                block_query += f"OR block_id='{block_id}' "
        block_query += ") AND title='A BLOCK'"

        query = f"""
        SELECT *
        FROM blocks 
        WHERE {block_query}
        """

        tdf = pd.read_sql(query,cnxn)
        #//*** A Block of Selected Rundown
        if len(tdf) == 0:
            print("================")
            print("NO A-Block Found")
            print("================")
            print(one_day_df)
            continue

        a_block = tdf["block_id"].values[0]
        #print("A Block:",a_block)

        #//******************************
        #//*** Get A-Block Story Titles 
        #//******************************
        query = f"""
        SELECT title_id
        --SELECT block_id,item_id,title_id
        FROM spots 
        WHERE block_id = '{a_block}'
        """

        title_id_list = pd.read_sql(query,cnxn)['title_id'].values


        #//****************************************************
        #//*** Get title_id, Story Slug, and Duration
        #//*** Only get Stories with Greater than 0 Duration
        #//****************************************************

        #//*** Build Single Query to get all Titles from Selected Rundown

        title_id_query = ""
        for title_id in title_id_list:
            if title_id == title_id_list[0]:
                title_id_query += f"title_id='{title_id}' "
            else:
                title_id_query += f"OR title_id='{title_id}' "

        query = f"""
        SELECT title_id,title_type_id,title,duration
        FROM titles 
        WHERE {title_id_query}
        """

        titles_df = pd.read_sql(query,cnxn)

        #//*** Remove Stories with Zero Duration
        #//*** This Removes stories with no text
        titles_df = titles_df[titles_df['duration'] > 0]

        #//*** Remove stories with the word tease in slug
        titles_df = titles_df[titles_df['title'].str.lower().str.contains('tease') == False]
        #print(titles_df)
        #//*** Get StoryBody Text
        story_id_list = titles_df['title_id'].values
        story_id_query = ""
        for story_id in story_id_list:
            if story_id == story_id_list[0]:
                story_id_query += f"TitleId='{story_id}' "
            else:
                story_id_query += f"OR TitleId='{story_id}' "

        query = f"""
        SELECT TitleId,StoryText
        FROM StoryContent
        WHERE {story_id_query}
        """

        text_df = pd.read_sql(query,cnxn)
        #print(text_df)

        #//*** Build Stories_df List of all Stories in show with Text and Title
        #//*** Merge titles_df and text_df 
        stories_df = titles_df.merge(text_df,left_on='title_id',right_on='TitleId')

        #//*** Delete the duplicate column
        del stories_df['TitleId']

        #//********************
        #//*** Clean the Text
        #//********************

        #//*** Remove Brackets [[ ]]
        stories_df['StoryText'] = stories_df['StoryText'].str.replace('\[\[\*\*\*.*?\*\*\*\]\]','\\n',regex=True)

        #//*** Remove paranthesis (( ))
        stories_df['StoryText'] = stories_df['StoryText'].str.replace('\(\(.*?\)\)','\\n',regex=True)

        #//*** Covert \r\n to \n
        stories_df['StoryText'] = stories_df['StoryText'].str.replace('\\r\\n','\\n',regex=True)

        #//*** Convert Multiple \n to single \n
        stories_df['StoryText'] = stories_df['StoryText'].str.replace('\\n\\W*\\n','\\n',regex=True)

        #//*** Delete leading \n
        stories_df['StoryText'] = stories_df['StoryText'].str.replace('^\\W*\\n','',regex=True)
        #print(stories_df.iloc[7]['StoryText'])

        #//*** Rename title column to storyslug
        stories_df.columns = ['storyslug' if x=='title' else x for x in list(stories_df.columns)]


        #//*** Start Date, Title, from the current rundown. This Column will have the same value for all rows.
        static_cols = ['start_date','title']

        for col in static_cols:
            stories_df[col] = loop_rundown[col]

        #//*** Shift the static_cols to the beginning of the column list for readability
        cols = (static_cols + list(stories_df.columns))[:(len(static_cols)*-1)]

        stories_df = stories_df[cols]


        #//*** Rename title column to storyslug
        stories_df.columns = ['rundown' if x=='title' else x for x in list(stories_df.columns)]


        #print(cols)
        #print(stories_df)
        qtr_df = pd.concat([qtr_df,stories_df],ignore_index=True)
        #//*** End Get Single Rundown Stories
    
    #//*** End Each Day of Rundowns

#//*** Remove Stories with 0 length Text
qtr_df['StoryText'] = qtr_df['StoryText'].astype(str)
qtr_df['length'] = qtr_df['StoryText'].apply(lambda x : len(x))
qtr_df = qtr_df[qtr_df['length'] > 20]

if 'title_type_id' in qtr_df.columns:
    del qtr_df['title_type_id']
    
if 'length' in qtr_df.columns:
    del qtr_df['length']
tdf = qtr_df
tdf['duration'] = (tdf['duration'] /1000).astype(int)    

#qtr_df.to_csv("2021_Q4_Stories.csv")  

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('2021_Q3_Stories.xlsx', engine='xlsxwriter')
qtr_df.to_excel(writer,sheet_name='sheet1')
writer.save()

qtr_df
#print("Done")

    
    
    
    

2021-07-01 / 2021-09-30
2021-07-02 / 2021-09-30
2021-07-03 / 2021-09-30
2021-07-04 / 2021-09-30
2021-07-05 / 2021-09-30
2021-07-06 / 2021-09-30
2021-07-07 / 2021-09-30
2021-07-08 / 2021-09-30
2021-07-09 / 2021-09-30
2021-07-10 / 2021-09-30
2021-07-11 / 2021-09-30
2021-07-12 / 2021-09-30
2021-07-13 / 2021-09-30
2021-07-14 / 2021-09-30
2021-07-15 / 2021-09-30
2021-07-16 / 2021-09-30
2021-07-17 / 2021-09-30
2021-07-18 / 2021-09-30
2021-07-19 / 2021-09-30
2021-07-20 / 2021-09-30
2021-07-21 / 2021-09-30
2021-07-22 / 2021-09-30
2021-07-23 / 2021-09-30
2021-07-24 / 2021-09-30
2021-07-25 / 2021-09-30
2021-07-26 / 2021-09-30
2021-07-27 / 2021-09-30
2021-07-28 / 2021-09-30
2021-07-29 / 2021-09-30
2021-07-30 / 2021-09-30
2021-07-31 / 2021-09-30
2021-08-01 / 2021-09-30
2021-08-02 / 2021-09-30
2021-08-03 / 2021-09-30
2021-08-04 / 2021-09-30
2021-08-05 / 2021-09-30
2021-08-06 / 2021-09-30
2021-08-07 / 2021-09-30
2021-08-08 / 2021-09-30
2021-08-09 / 2021-09-30
2021-08-10 / 2021-09-30
2021-08-11 / 202

,start_date,rundown,title_id,storyslug,duration,StoryText
7,2021-07-01,5PM Weekday,31565204,5PCOLDOPEN 1,8,[[QUOTE: are you playing pop music to drown ou...
8,2021-07-01,5PM Weekday,31565217,5P HELLO FRAMES,7,"Good Evening, I'm Dan Ashley.\nand I'm Ama Da..."
12,2021-07-01,5PM Weekday,31579703,5P FIREWORKS CONCERNS PKG,78,{***PKG***}\nAs Bay Area families prepare to c...
13,2021-07-01,5PM Weekday,31579811,5P BAYVIEW INTERSECTION FLS INT,14,One of the things that neighbors told me is th...
14,2021-07-01,5PM Weekday,31579835,5P BAYVIEW INTERSECTION PKG,107,Neighbors began taking pictures after complain...
...,...,...,...,...,...,...
25870,2021-09-30,After The Game,33036449,ATG OREGON VS STANFORD OC,104,onto college football...stanford hosting 3rd r...
25873,2021-09-30,After The Game,33036454,ATG WSU VS CALIFORNIA OC,38,"cal lost a heartbreaker last week, fumbling at..."
25877,2021-09-30,After The Game,33036496,ATG PADRES VS GIANTS OC,95,the giants have won a franchise record-tying 1...
25879,2021-09-30,After The Game,33036500,ATG PADRES VS GIANTS OC,27,"so, the giants need to beat the padres tomorro..."


In [508]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('2021_Q4_Stories.xlsx', engine='xlsxwriter')
qtr_df.to_excel(writer,sheet_name='sheet1')
writer.save()
#qtr_df['duration'] = (qtr_df['duration']/1000).astype(int)
#qtr_df.to_csv("2021_Q4_Stories.csv")

FileCreateError: [Errno 13] Permission denied: '2021_Q4_Stories.xlsx'

0                                                " \r\n"
1                                                " \r\n"
2                                                " \r\n"
3                                                " \r\n"
4                                                " \r\n"
                             ...                        
453                                              " \r\n"
454    " \r\n[[***WEBPC***]]\r\nYou can get the lates...
455                                              " \r\n"
456                                              " \r\n"
457    " \r\n[[***VO***]]\r\n((CG:  00-END BANNER))\r...
Name: StoryText, Length: 458, dtype: object

In [324]:
stories_df = titles_df.merge(text_df,left_on='title_id',right_on='TitleId')
del stories_df['TitleId']
stories_df['StoryText'] = stories_df['StoryText'].str.replace('\[\[.+\]\]','',regex=True)
stories_df['StoryText'] = stories_df['StoryText'].str.replace('\(\(.+\)\)','',regex=True)
stories_df['StoryText'] = stories_df['StoryText'].str.replace('\\r\\n','\\n',regex=True)
stories_df['StoryText'] = stories_df['StoryText'].str.replace('\\n\\W*\\n','\\n',regex=True)
stories_df['StoryText'] = stories_df['StoryText'].str.replace('^\\W*\\n','',regex=True)
print(stories_df.iloc[7]['StoryText'])
stories_df

<<"HEALTH EXPERTS AROUND THE WORLD ARE UNITED IN AN EFFORT TO ATTAIN MORE INFORMATION REGARDING THE OMICRON VARIANT.
IT'S ALREADY BEEN DETECTED IN EIGHTEEN COUNTRIES ACROSS THE GLOBE
SOT: DR. COLLINS: "we are quickly trying to figure out in south Africa is this, in fact, more contagious than other variants? It does look like it's spreading quite quickly there"
DR. FRANCIS COLLINS OF THE NATIONAL INSTITUTES OF HEALTH SAYS THE BIGGEST QUESTION SURROUNDING OMICRON IS IF THE AVAILABLE VACCINES WILL OFFER THE SAME LEVEL OF PROTECTION AGAINST IT -- A QUESTION THAT CAN'T BE ANSWERED RIGHT NOW.
SOT: DR. COLLINS: "That is not something you can do overnight no matter how hard you try so people need to kind of hang on here for a couple of weeks"
PRESIDENT BIDEN SPEAKING TO THE NATION MONDAY SAYS THE VARIANT IS A CAUSE FOR CONCERN BUT NOT A CAUSE FOR PANIC
SOT: BIDEN:  "If people are vaccinated and wear their mask, there's no need for a  lockdown,"
THE CDC STRENGTHENING ITS  RECCOMENDATIONS -- NOW

,title_id,title_type_id,title,duration,StoryText
0,33958087,1020,COLDOPEN,9415,\tOakland's police chief-- once again-- pleadi...
1,33958093,4,ANCHOR INTRO,6384,"\tHello, I'm Reggie Aqui.\n\tAnd I'm Kristen S..."
2,33958100,4,TOSS WEATHER,4674,BRACE FOR HEAT ... METEOROLOGIST MIKE NICCO SA...
3,33958112,4,TEASE 1,10203,"JUST AHEAD, THE REASON ADELE FANS ARE LOSING ..."
4,33990342,4,11A HOLIDAY VARIANT OC,36805,\tWe know a lot of you are wondering how the n...
5,33990381,4,11A KEVIN NISHITA OC,39330,"\tThis morning, police are hoping someone has ..."
6,33990511,1020,11A SCHOOL VARIANT VO,48112,Even though the omicron variant hasn't been de...
7,33990711,1020,11A OMICRON LATEST PKG,123462,"<<""HEALTH EXPERTS AROUND THE WORLD ARE UNITED ..."
8,33990819,4,11A GRAND SHOOTING,52765,\tThe grief-stricken family of Oakland's lates...
9,33992129,1020,11A OMICRON STOCKS,20634,TAKE A LOOK AT THE BIG BOARD from just a littl...


In [7]:

query = f"""
SELECT TitleId,StoryText
FROM StoryContent
LIMIT 1
"""
query = f"""
SELECT TOP 1 *
FROM StoryContent

"""
df = pd.read_sql(query,cnxn)

In [13]:
hunt_db("36596420")

Titles_ExFields Title_id 1
TitleDataDictString TitleId 4
TitleDataDictText TitleId 1
titles_extend title_id 1
ItemsToBeIndexed Item_id 1
titles_in_class title_id 1
objects_in_titles Object_id 1
titles_references title_id 1
titles_mix title_id 1
CreatedModifiedItems Item_id 1
ObjectStatus ObjectId 1
titles title_id 1
StatusLog ObjectId 1
Titles_ExFields -- Title_id
Title_id, Language_id, Topic, VersionComment, 
Table
( 36596420, 99, None, Title Created,  )
TitleDataDictString -- TitleId
DataField_Id, TitleId, ValueOrder, StrValue, 
Table
( 93, 36596420, 1, 5b9e944268ee9,  )
( 94, 36596420, 1, kgo.pcr2.ovd.mos,  )
( 1178, 36596420, 1, --,  )
( 1232, 36596420, 1, --,  )
TitleDataDictText -- TitleId
DataField_Id, TitleId, ValueOrder, TextValue, 
Table
( 82, 36596420, 1, <mos><mosID>kgo.pcr2.ovd.mos</mosID><itemID>0</itemID><mosAbstract>v3a Acuity OD ESU - Ross Video - Transition: 1_CUT _CLRG1 - Audio AFV: Disabled - Custom Controls: OD ESU (Bank:31 Btn:31); FTB BUG_OUT (Bank:31 Btn:2); CLR

In [14]:
hunt_db("36596418")

Titles_ExFields Title_id 1
TitleDataDictInteger TitleId 43
TitleDataDictString TitleId 29
TitleDataDictDateTime TitleId 2
TitleDataDictText TitleId 2
TitleDataDictPeriod TitleId 1
titles_extend title_id 1
ItemsToBeIndexed Item_id 1
spots title_id 1
titles_in_class title_id 1
objects_in_titles Title_id 7
titles_references title_id 1
titles_mix title_id 1
CreatedModifiedItems Item_id 1
StoryContent TitleId 1
ObjectStatus ObjectId 1
titles title_id 1
StatusLog ObjectId 1
Titles_ExFields -- Title_id
Title_id, Language_id, Topic, VersionComment, 
Table
( 36596418, 99,  , None,  )
TitleDataDictInteger -- TitleId
DataField_Id, TitleId, ValueOrder, NumValue, 
Table
( 67, 36596418, 1, 0,  )
( 68, 36596418, 1, 1,  )
( 69, 36596418, 1, 0,  )
( 71, 36596418, 1, 1,  )
( 73, 36596418, 1, 1,  )
( 76, 36596418, 1, 0,  )
( 78, 36596418, 1, 0,  )
( 114, 36596418, 1, 29557076,  )
( 114, 36596418, 2, 29842610,  )
( 114, 36596418, 3, 29843174,  )
( 647, 36596418, 1, 1,  )
( 648, 36596418, 1, 1,  )
( 649, 3

91687 OverDrive Shots in the DB

In [167]:
query = f"""
SELECT  TOP 30000 title_id
FROM Titles
WHERE title_type_id=1046 
ORDER BY start_date ASC
"""
df = pd.read_sql(query,cnxn)
title_list = list(df['title_id'])


In [168]:
def build_title_query(field,il):
    out = ""
    
    for x in il:
        if x != il[-1]:
            out += f"{field}={x} OR "
        else:
            out += f"{field}={x}"
    return out
query_titles = build_title_query("TitleId",title_list)

In [169]:

query = f"""
SELECT  TOP {len(title_list)} TextValue
FROM TitleDataDictText 
WHERE DataField_Id = 82 AND ({query_titles})

"""
df = pd.read_sql(query,cnxn)
df

,TextValue
0,<mos><mosID>kgo.pcr2.ovd.mos</mosID><itemID>2<...
1,<mos><mosID>kgo.pcr2.ovd.mos</mosID><mosAbstra...
2,<mos><mosID>kgo.pcr2.ovd.mos</mosID><mosAbstra...
3,<mos><mosID>kgo.pcr2.ovd.mos</mosID><mosAbstra...
4,<mos><mosID>kgo.pcr2.ovd.mos</mosID><mosAbstra...
...,...
29995,<mos><mosID>kgo.pcr2.ovd.mos</mosID><itemID>0<...
29996,<mos><mosID>kgo.pcr2.ovd.mos</mosID><mosAbstra...
29997,<mos><mosID>kgo.pcr2.ovd.mos</mosID><itemID>0<...
29998,<mos><mosID>kgo.pcr2.ovd.mos</mosID><itemID>0<...


In [170]:
import re
bn_list = []
for row in df.iterrows():
    
    #display(row[1]['TextValue'])
    ccs = re.findall("<customControl>.+?</customControl>", row[1]['TextValue'])
    if len(ccs) == 0:
        continue
    #print(re.findall("<mosAbstract>.+?</mosAbstract>", row[1]['TextValue']))
    for cc in ccs:
        #print("\t",len(ccs),cc)
        bank = re.search("<bank>.+?</bank>",cc)
        button = re.search("<button>.+?</button>",cc)
        #bn = f"BANK_{bank.group().replace('<bank>','').replace('</bank>','') }_BUTTON_{button.group().replace('<button>','').replace('</button>','') }"
        bn = (int(bank.group().replace('<bank>','').replace('</bank>','')),int(button.group().replace('<button>','').replace('</button>','') ))
        bn_list.append(bn)

bn_list.sort()    
print(bn_list)
#<mosAbstract>v2v Cue Open - BABBA - Edu1 - Ross Video - Audio AFV: Video Only - K2 POOL Clip: O_DAY_BBA_EDUCLASS1</mosAbstract>    

[(1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 37), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 38), (1, 39), (1, 39), (1, 39), (1, 39), (1, 42), (1, 42), (1, 42), (1, 42), (1, 42), (1, 42), 

In [184]:
bb = {}
for bn in bn_list:
    key = f"BANK_{bn[0]}_BTN_{bn[1]}"
    
    if key not in bb.keys():
        bb[key] = 1
    else:
        bb[key] += 1

print("===========================================================")
print(f"Custom Control Count From last {len(title_list)} Recently Used Shots")
print("===========================================================")
for key,value in bb.items():
    spacer = (14 - len(key)) * " "
    spacer_1 = (5 - len(str(value))) * " "
    print(f"{key}:{spacer} {value}{spacer_1} Instances - {round((value / len(bn_list)+.000001),4)*100 }%")

Custom Control Count From last 30000 Recently Used Shots
BANK_1_BTN_37:  64    Instances - 0.25%
BANK_1_BTN_38:  37    Instances - 0.13999999999999999%
BANK_1_BTN_39:  4     Instances - 0.02%
BANK_1_BTN_42:  12    Instances - 0.05%
BANK_2_BTN_25:  3     Instances - 0.01%
BANK_3_BTN_2:   76    Instances - 0.29%
BANK_3_BTN_3:   27    Instances - 0.1%
BANK_3_BTN_7:   14    Instances - 0.05%
BANK_3_BTN_9:   7     Instances - 0.03%
BANK_3_BTN_10:  61    Instances - 0.22999999999999998%
BANK_3_BTN_13:  2     Instances - 0.01%
BANK_3_BTN_14:  12    Instances - 0.05%
BANK_3_BTN_17:  66    Instances - 0.25%
BANK_3_BTN_18:  71    Instances - 0.27%
BANK_3_BTN_22:  1     Instances - 0.0%
BANK_3_BTN_24:  108   Instances - 0.41000000000000003%
BANK_3_BTN_27:  360   Instances - 1.38%
BANK_3_BTN_28:  37    Instances - 0.13999999999999999%
BANK_3_BTN_29:  1772  Instances - 6.800000000000001%
BANK_3_BTN_30:  31    Instances - 0.12%
BANK_3_BTN_31:  3270  Instances - 12.55%
BANK_3_BTN_32:  4     Instances